In [1]:
import polaris
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from qsarcons.lazy import LazyML
from qsarcons.consensus import RandomSearchRegressor, SystematicSearchRegressor, GeneticSearchRegressor

/storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from polaris.hub.client import PolarisHubClient
# with PolarisHubClient() as client:
#     client.login()

### 1. Load data

In [3]:
# Load the benchmark from the Hub
benchmark = polaris.load_benchmark("polaris/adme-fang-solu-1")

# Get the train and test data-loaders
data_train, data_test = benchmark.get_train_test_split()
data_train, data_test = data_train.as_dataframe(), data_test.as_dataframe()

smi_train, prop_train = data_train["smiles"].to_list(), data_train["LOG_SOLUBILITY"].to_list()

data_train, data_val = train_test_split(data_train, test_size=0.2, random_state=42)

[2025-10-06 07:14:45] INFO     The version of Polaris that was used to create the artifact (0.0.0)  ]8;id=666880;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=234449;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.13.0).                                                                           

                      INFO     To verify the checksum, we need to recompute it. This can be slow    ]8;id=292307;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=780069;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py#67\67]8;;\
                               for large datasets.                                                                 

[07:14:45]  Success: Fetching dataset                                                                 ]8;id=192660;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=185980;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

                      INFO     The version of Polaris that was used to create the artifact (0.0.0)  ]8;id=957055;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py\_artifact.py]8;;\:]8;id=59990;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/_artifact.py#96\96]8;;\
                               is different from the currently installed version of Polaris                        
                               (0.13.0).                                                                           

                      WARNING  No checksum to verify against. Specify either the md5sum parameter   ]8;id=898014;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py\_checksum.py]8;;\:]8;id=316851;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/mixins/_checksum.py#60\60]8;;\
                               or store the checksum in the dataset.md5sum attribute.                              

            Success: Fetching benchmark                                                               ]8;id=803337;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py\context.py]8;;\:]8;id=922660;file:///storage/dmitry/miniforge3/envs/qsarmil/lib/python3.10/site-packages/polaris/utils/context.py#53\53]8;;\

### 2. Build multiple 2D models

In [ ]:
data_test["LogS"] = [0 for i in data_test.index]

lazy_ml = LazyML(task="regression", output_folder="logs_bench", verbose=True)
lazy_ml.run(data_train, data_val, data_test)

Total models to build: 152


 55%|████████████████████████████████████████████▊                                     | 83/152 [06:25<06:10,  5.37s/it]

### 3. Build model consensus

In [ ]:
metric = "auto"
cons_size = "auto"

In [ ]:
cons_methods = [
    ("Best", SystematicSearchRegressor(cons_size=1, metric=metric)),         
    ("Random", RandomSearchSystematicSearchRegressor(cons_size=cons_size, n_iter=1000, metric=metric)),       
    ("Systematic", SystematicSearchSystematicSearchRegressor(cons_size=cons_size, metric=metric)),
    ("Genetic", GeneticSearchSystematicSearchRegressor(cons_size=cons_size, n_iter=50, pop_size=50, mut_prob=0.2, metric=metric))
]

In [ ]:
# load model predictions
df_val = pd.read_csv("logs_bench/val.csv")
df_test = pd.read_csv("logs_bench/test.csv")

# skip first two columns (smiles and true property value)
x_val, true_val = df_val.iloc[:, 2:], df_val.iloc[:, 1]
x_test, true_test = df_test.iloc[:, 2:]

In [ ]:
for name, cons_searcher in cons_methods:
    
    # run search
    best_cons = cons_searcher.run(x_val, true_val)
    
    # make val and test predictions
    pred_val = cons_searcher._consensus_predict(x_val[best_cons])
    pred_test = cons_searcher._consensus_predict(x_test[best_cons])
    
    # write prediction accuracy metric
    df_val[name] = pred_val
    df_test[name] = pred_test